<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/langchain/rag/Langchain_RAG_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ask A Book A Question

QA with Langchain and use your own data to build a qa or RAG tool

https://huggingface.co/blog/getting-started-with-embeddings

#### QA

https://huggingface.co/tasks/question-answering

#### RAG

https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/

#### VectorStores Langchain

https://python.langchain.com/docs/integrations/vectorstores

#### Chroma

https://python.langchain.com/docs/integrations/vectorstores/chroma


In [ ]:
!pip install langchain langchain-community tiktoken -q
!pip install -U accelerate -q
! pip install -U unstructured numpy -q
! pip install pypdf
!pip install ctransformers -q
!pip install chainlit -q
! pip install sentence-transformers -q
! pip install chromadb -q

In [1]:

from google.colab import output
output.enable_custom_widget_manager()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import CTransformers

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import chainlit as cl

In [6]:

from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
import glob
import re
path_docs  = "/content/drive/MyDrive/data (1)/documents"
docs = glob.glob(f"{path_docs}/*")

In [15]:
for doc in docs:
  print(doc)

/content/drive/MyDrive/data (1)/documents/Attention Is All You Need.pdf
/content/drive/MyDrive/data (1)/documents/Training compute-optimal LLM.pdf
/content/drive/MyDrive/data (1)/documents/BloombergGPT.pdf
/content/drive/MyDrive/data (1)/documents/BLOOM.pdf
/content/drive/MyDrive/data (1)/documents/Scaling Laws for Neural Language Models.pdf
/content/drive/MyDrive/data (1)/documents/What Language Model Architecture and Pretraining Objective Work Best for Zero-Shot Generalization.pdf
/content/drive/MyDrive/data (1)/documents/LLaMA Open and Efficient Foundation Language Models.pdf
/content/drive/MyDrive/data (1)/documents/Language Models are Few-Shot Learners.pdf
/content/drive/MyDrive/data (1)/documents/FLAN Fine-tuned LAnguage Net.pdf
/content/drive/MyDrive/data (1)/documents/SuperGLUE.pdf
/content/drive/MyDrive/data (1)/documents/GLUE.pdf
/content/drive/MyDrive/data (1)/documents/ROUGE.pdf
/content/drive/MyDrive/data (1)/documents/MEASURING MASSIVE MULTITASK Language Understanding MML

In [19]:
# https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4500, chunk_overlap=0)

In [20]:
texts=[]
for doc in docs:

  # Load the book
  loader = PyPDFLoader(doc)
  pages = loader.load()
  # Combine the pages, and replace the tabs with spaces
  text = ""

  for page in pages:
      text += page.page_content

  text = text.replace('\t', ' ')
  #remove special chars
  res = re.sub('[!,*)@#%(&$_?.^]', '', text)
  print(f"document {doc} has a length of {len(res)}")
  # Split the book into chunks
  docums = text_splitter.create_documents([res])
  # Add the chunks to the list
  for doc in docums:
    texts.append(doc)

document /content/drive/MyDrive/data (1)/documents/Attention Is All You Need.pdf has a length of 38245
document /content/drive/MyDrive/data (1)/documents/Training compute-optimal LLM.pdf has a length of 89814
document /content/drive/MyDrive/data (1)/documents/BloombergGPT.pdf has a length of 193655
document /content/drive/MyDrive/data (1)/documents/BLOOM.pdf has a length of 190448
document /content/drive/MyDrive/data (1)/documents/Scaling Laws for Neural Language Models.pdf has a length of 83731
document /content/drive/MyDrive/data (1)/documents/What Language Model Architecture and Pretraining Objective Work Best for Zero-Shot Generalization.pdf has a length of 84553
document /content/drive/MyDrive/data (1)/documents/LLaMA Open and Efficient Foundation Language Models.pdf has a length of 84083
document /content/drive/MyDrive/data (1)/documents/Language Models are Few-Shot Learners.pdf has a length of 228343
document /content/drive/MyDrive/data (1)/documents/FLAN Fine-tuned LAnguage Net

In [21]:
# Let's see how many small chunks we have
print (f'Now you have {len(texts)} documents')

Now you have 734 documents


In [22]:
# load it into Chroma
vectorstore = Chroma.from_documents(texts, embeddings)

In [23]:
query = "What is a transformer?"
docs = vectorstore.similarity_search(query)

In [25]:
# Here's an example of the first document that was returned
for doc in docs:
    print (f"{doc.page_content}\n")
    print("-"*50)
    print("\n")

niques discussed in this survey can be applied
to all neural networks but some are speciﬁcally
designed to take advantage of the Transformer
architecture Vaswani et al 2017 Given that
Transformers are the largest neural networks ever
trained these methods are particularly valuable
Thus we present a brief overview of the Trans-
former to provide context for these techniques
The core building block of the Transformer ar-
chitecture consists of multi-head attention MHA
followed by a fully-connected network FFN as
illustrated in Figure 1 Both attention and fully-
connected layers incorporate residual connections
He et al 2016 and Layer Normalization Ba
et al 2016 to improve trainability
The heart of the Transformer is attention oper-
ation Following the NamedTensor notation Chi-
ang et al 2021 it can be described as
Att:Rkey×Rseq×key×Rseq×val→Rval
AttQKV  =
softmax
seqQ⊙
keyK
√
|key|
⊙
seqV
Qx =x·WQ+bk
Kx =x·WK+bq
Vx =x·WV+bv
WQ WK∈Rinput×keyWV∈Rinput×val
bQ bK∈Rkey bV∈Rval
Figure 1: B